In [15]:
from bs4 import BeautifulSoup
import requests
import time
import datetime
import re
import csv
import pandas as pd
import re
import os
from __future__ import division

import smtplib
from dateutil import parser
from datetime import date

import gspread
import google.auth
from oauth2client.service_account import ServiceAccountCredentials
from googleapiclient.discovery import build
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
from googleapiclient.errors import HttpError


In [27]:
########################################################
########################################################


# contador de execucao
start_time = time.time()



# google

scopes = ["https://www.googleapis.com/auth/spreadsheets",
          "https://www.googleapis.com/auth/drive"
]


# principal
def main_cassio(lista):
    spreadsheet_id = '1R504OJcC8gFd-b62WvENz8p-crcLtl2qGbp3tg3A_vY'  # this is part of the url of google

    credentials = ServiceAccountCredentials.from_json_keyfile_name("C:\\Users\\Cássio Berger\\Documents\\Programação\\_scripts - nao mexer\\acoes-b3-indicadores-fund-key.json", scopes=scopes)
    service = build('sheets', 'v4', credentials=credentials)
    
    # definindo onde ira escrever (aba!intervalo celulas)
    service.spreadsheets().values().append(
        spreadsheetId=spreadsheet_id,
        range="[script] ATUAL [webscraping]!A14:Z999",
        body={
            "majorDimension": "ROWS",
            "values": lista
        },
        valueInputOption="USER_ENTERED"
    ).execute()


    
    
def main_davi(lista):
    spreadsheet_id = '1R504OJcC8gFd-b62WvENz8p-crcLtl2qGbp3tg3A_vY'  # this is part of the url of google

    credentials = ServiceAccountCredentials.from_json_keyfile_name("C:\\Users\\Cássio Berger\\Documents\\Programação\\_scripts - nao mexer\\acoes-b3-indicadores-fund-key.json", scopes=scopes)
    service = build('sheets', 'v4', credentials=credentials)
    
    # definindo onde ira escrever (aba!intervalo celulas)
    service.spreadsheets().values().append(
        spreadsheetId=spreadsheet_id,
        range="[DAVI][script] ATUAL [webscraping]!A4:Z999",
        body={
            "majorDimension": "ROWS",
            "values": lista
        },
        valueInputOption="USER_ENTERED"
    ).execute()

    

########################################################    
    
    
# seta as credenciais para acesso; usando conta de serviço registrada; seta o alvo a ser escrito
# Source: https://developers.google.com/sheets/api/quickstart/python
def get_or_create_credentials(scopes):
    credentials = None
    if os.path.exists('token.pickle'):
        with open('token.pickle', 'rb') as token:
            credentials = pickle.load(token)
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file('C:\\Users\\Cássio Berger\\Documents\\Programação\\_scripts - nao mexer\\acoes-b3-indicadores-fund-key.json', scopes)
            credentials = flow.run_local_server(port=0)
            print("\n\nAchou json")
        with open('token.pickle', 'wb') as token:
            pickle.dump(credentials, token)
    return credentials


########################################################

# combina permissão com destino
def get_values(spreadsheet_id, range_name):
  # pylint: disable=maybe-no-member
  creds = ServiceAccountCredentials.from_json_keyfile_name("C:\\Users\\Cássio Berger\\Documents\\Programação\\_scripts - nao mexer\\acoes-b3-indicadores-fund-key.json", scopes=scopes)
  try:
    service = build("sheets", "v4", credentials=creds)

    result = (
        service.spreadsheets()
        .values()
        .get(spreadsheetId=spreadsheet_id, range=range_name)
        .execute()
    )
    rows = result.get("values", [])
    print(f"{len(rows)} linhas encontradas na tabela")
    return result
  except HttpError as error:
    print(f"Erro: {error}")
    return error



########################################################
########################################################
########################################################
########################################################



# getting initial data pattern url, 10 pages loop

lista_links=[]
lista_nomes=[]
lista_nomes_links=[]


############################

# loop para pegar ações do cassio e depois davi

for cassio_ou_davi in range(2):
    # pegando codigos de ações dinamicamente da planilha
    if cassio_ou_davi == 0:
        # cassio
        valores=get_values("1R504OJcC8gFd-b62WvENz8p-crcLtl2qGbp3tg3A_vY", "[script]lista de acoes ATUAL!B2:B99")

    else:
        # davi
        valores=get_values("1R504OJcC8gFd-b62WvENz8p-crcLtl2qGbp3tg3A_vY", "[DAVI][script]lista de acoes!B2:B99")
        #print("\n PEGOU valores davi")
        #print("valores ta planilha: ", str(valores.values))

    lista_acoes_planilha = []
    for i in valores.values():
        lista_acoes_planilha.append(i)


    # tirando lista dentro de lista [[]] -> []
    nomes_acoes = [
        acao
        for xs in lista_acoes_planilha[2]
        for acao in xs
    ]
    #print(nomes_acoes)



    ############################

    # lista de ações a serem pesquisadas (fonte PLANILHA)
    #https://docs.google.com/spreadsheets/d/1R504OJcC8gFd-b62WvENz8p-crcLtl2qGbp3tg3A_vY/edit#gid=610897588

    #nomes_acoes=['GOAU4','AGRO3','CMIN3','ABCB4','TAEE4','SAPR11','SAPR4','VULC3','ALOS3','TCSA3']
    #nomes_acoes=['ABCB4']


    # padrao de links para ações; constroi link completo a partir da lista
    acoes_urls = [f"https://www.fundamentus.com.br/detalhes.php?papel={l}" for l in nomes_acoes]
    



    # laço

    #for acao in acoes_urls:
    for acao in nomes_acoes:
        #print(acao)
        #print(acoes_urls[0])
        try:
            # criando url acao
            URL = f"https://www.fundamentus.com.br/detalhes.php?papel={acao}"

            # criando url proventos da acao; removendo espacos em branco
            acao_sem_espaco= re.sub(' ','', str(acao))
            URL_proventos = f"https://www.fundamentus.com.br/proventos.php?papel={acao_sem_espaco}&tipo=2"


            # indices acoes
            headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
            page = requests.get(URL, headers=headers)
            souptext = BeautifulSoup(page.text)


            # pega todas as tabelas; a que precisa usar é a 3, logo usa-se a 2 abaixo
            tabela_indices=souptext.select('table.w728')
            tabela_certa=tabela_indices[2]



            ########################################################
            ########################################################
            ########################################################

            # proventos


            # acesso pagina de proventos
            page = requests.get(URL_proventos, headers=headers)
            soupproventos = BeautifulSoup(page.text)

            # pegando tabela com dados de proventos
            tabela_proventos = soupproventos.find_all('td')
            #print(tabela_proventos)

            # definindo o ano passado
            today = datetime.date.today()
            ano = today.year
            ano=ano-1

            cont=0

            # lista para pegar primeiro e segundo indices, pulando cada linha
            data_prov=[]
            data_prov=list(tabela_proventos[0::5])

            valor_prov=[]
            valor_prov=list(tabela_proventos[1::5])


            # contador de proventos
            numero_prov=0

            prov=[]
            lista_soma_proventos_por_acao=[]

            for c in range(0, len(nomes_acoes)):
                if str(ano) in str(data_prov[c]):
                    #print("\nDEBUG")
                    #print(data_prov[i])

                    # regex para formatar
                    linha_prov_val = re.sub('<td>','', str(valor_prov[c]))
                    linha_prov_val= re.sub('</td>','', str(linha_prov_val))
                    linha_prov_val= re.sub(',','.', str(linha_prov_val))
                    #print("\nLinha valor")
                    #print(linha_prov_val)
                    prov.append(linha_prov_val)

                    numero_prov+=1

            # somando todos valores de proventos ano passado
            soma_proventos=sum(map(float, prov))


            ############################
            # print para verificar

            #if(numero_prov!=0):
            #    print("\n---------Lista de valores proventos ano passado ", prov)
            #    print("\n---------soma_proventos: ", soma_proventos)
            #    print("\n---------Numero de pagamentos de proventos: ", numero_prov,"\n") 



           ########################################################
           ########################################################
           ########################################################


            # separando informações relevantes (por linhas)
                #P/L
                #Margem Ebit % 
                #P/VP 
                #DY %
                #ROE %
                #Margem Liquida %


            # tabela para manipulação
            tabela_celula=[]

            # entrando nas linhas e colunas da tabela desejada
            for linha in tabela_certa:
                for coluna in linha:            

                    # preparando dados e limpando
                    # substituindo linha e % para usar
                    result=re.sub(' +','', str(coluna))
                    result=re.sub('\n','', str(coluna))
                    result=re.sub('%','', str(result))
                    #print(result)

                    # separando linhas com dados, removendo dos labels etc
                    result = re.search('<span class="txt">(.*)', str(result))
                    #print(result)

                    # apenas para manipulação, se vazia pula
                    if(not result):
                        pass
                    # adiciona na tabela, caso estiver com alguma valor
                    else:
                        tabela_celula.append(result)
                        #print(tabela_celula)


            ############################


            # Linhas para utilizar; abaixo
            #P/L 3,4
            #Margem Ebit % 20,21
            #P/VP 8,9
            #DY % 38,39
            #ROE % 40,41
            #Margem Liquida % 25,26


            # print para verificar dados com numeração
            #for i in range(0,len(tabela_celula)):
            #    print("\n")
            #    print(i, str(tabela_celula[i]))


           ############################


            # tabelas finais com as linhas
            tabela_final=[]
            tabela_indices=[]
            tabela_multiplos=[]


            #for i in [3,4,8,9,20,21,25,26,38,39,40,41]:
            for i in [3,4,20,21,8,9,38,39,40,41,25,26]:
                # substituindo com expressoes regulares, deixando apenas indices e multiplo
                tabela_celula[i]=re.sub(r'<re.Match object; span=(.*)\, match=\'<span class="txt">','', str(tabela_celula[i]))
                tabela_celula[i]=re.sub(r'</span></td>\'>','', str(tabela_celula[i]))

                #print(tabela_celula[i])

                # separando valores dos indices, apenas manipulação; 
                #se contem ',' é numero, se nao é string
                # se contem '-', recebe valor '0,0'
                # e se contem
                if(',' in tabela_celula[i]):
                    tabela_multiplos.append(tabela_celula[i])

                elif('-' in tabela_celula[i]):
                    tabela_multiplos.append('0,0')

                elif('-' not in tabela_celula[i]):
                    tabela_indices.append(tabela_celula[i])


            ############################

            # esperando 1 segundo para enviar nova requisição
            time.sleep(1)



            ############################ inserindo data e ticker        
            # inserindo dia no inicio e convertendo formato
            hoje = date.today()
            new_hoje = hoje.strftime("%d/%m/%Y")
            #print(new_hoje)

            # inserindo ticker realizado
            tabela_multiplos.insert(0, acao)        
            tabela_indices.insert(0, "Acao")   

            # inserindo dia realizado
            tabela_multiplos.insert(1, new_hoje)        
            tabela_indices.insert(1, "Data coleta")   

            # inserindo quantidade de pagamentos feitos ano passado
            tabela_multiplos.append(numero_prov)        
            tabela_indices.append("Numero de proventos ano passado")  

            
            # inserindo dy_por_pagamento; quanto é pago por dividendo cada vez que ela paga
            # troca , por ., e depois converte para float
            tabela_multiplos[5] = tabela_multiplos[5].replace(',', '.')
            
            if float(tabela_multiplos[8]) !=0:
                dy_por_pag = float(tabela_multiplos[5])/float(tabela_multiplos[8])
            else:
                dy_por_pag = 0
            
            tabela_multiplos.append(dy_por_pag)
            
            
            
            ############################


            # print verficação
            #print(len(tabela_indices))
            #print(tabela_indices)
            #print(len(tabela_multiplos))
            #print(tabela_multiplos)


            ############################ criando e salvando lista em dataframe
            
            acoes_scraping = pd.DataFrame(tabela_multiplos)


            # transposing colunas e linhas, para visualização

            acoes_scraping=acoes_scraping.T

            #acoes_scraping.to_csv("acoes_scraping.csv", mode='a', header=False,index=False)
            #print(f'@@@@@@@@@@ Linha escrita no arquivo, {acao} @@@@@@@@@@')
            



            ############################ chama função google sheet, passando cada linha (acao, multiplos) para salvar

            if cassio_ou_davi == 0:
                main_cassio([tabela_multiplos])
                print("\nCCCCCCCCCCCCCCCCCCCCCCCCC     LISTA ações CASSIO salva, Ação: ", acao)

            else:
                main_davi([tabela_multiplos])
                print("\nDDDDDDDDDDDDDDDDDDDDDDDDD     LISTA ações DAVI salva, Ação: ", acao)
                pass
            

        except:
            pass
        

    
#print('\n\n ############################ Ordem dos indicadores ############################')
#print('\n', tabela_indices)
#print(nomes_acoes)
print('\n\n ############################ Script Concluido ############################')
print("--- %s segundos ---" % (time.time() - start_time))

10 linhas encontradas na tabela
GOAU4

CCCCCCCCCCCCCCCCCCCCCCCCC     LISTA ações CASSIO salva, Ação:  GOAU4
AGRO3

CCCCCCCCCCCCCCCCCCCCCCCCC     LISTA ações CASSIO salva, Ação:  AGRO3
CMIN3

CCCCCCCCCCCCCCCCCCCCCCCCC     LISTA ações CASSIO salva, Ação:  CMIN3
ABCB4

CCCCCCCCCCCCCCCCCCCCCCCCC     LISTA ações CASSIO salva, Ação:  ABCB4
TAEE4 

CCCCCCCCCCCCCCCCCCCCCCCCC     LISTA ações CASSIO salva, Ação:  TAEE4 
SAPR11

CCCCCCCCCCCCCCCCCCCCCCCCC     LISTA ações CASSIO salva, Ação:  SAPR11
SAPR4

CCCCCCCCCCCCCCCCCCCCCCCCC     LISTA ações CASSIO salva, Ação:  SAPR4
VULC3

CCCCCCCCCCCCCCCCCCCCCCCCC     LISTA ações CASSIO salva, Ação:  VULC3
ALOS3

CCCCCCCCCCCCCCCCCCCCCCCCC     LISTA ações CASSIO salva, Ação:  ALOS3
TCSA3

CCCCCCCCCCCCCCCCCCCCCCCCC     LISTA ações CASSIO salva, Ação:  TCSA3
8 linhas encontradas na tabela
ITSA4
B3SA3
GGBR4
VIVT3
PETR4
CMIG4
CPLE6
GOAU4


 ############################ Script Concluido ############################
--- 49.28375267982483 segundos ---
